In [1]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

df = pd.read_csv('../datasets/3_data_paragraph_labelled_prepared.csv', encoding='utf-8', encoding_errors='replace')

C:\Users\12406\AppData\Local\Temp\ipykernel_9608\216391889.py:7: DtypeWarning: Columns (1,8,10,11,13,14,15,17,18,27,37,38,63,65,81,114,148,158,166,167) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../datasets/3_data_paragraph_labelled_prepared.csv', encoding='utf-8', encoding_errors='replace')


In [2]:
len(df)

128213

In [3]:
df.head()

,claim,label,question,answer,positive_evidence_text,positive_evidence_title,negative_evidence_text,negative_evidence_title,split,dataset,...,evidence_type,is_auth,evidence_id,evidence_label,article,evidence,entropy,Misinfo_type,speaker,veracity
0,NaN,unreliable,NaN,NaN,NaN,NaN,NaN,NaN,NaN,fake_news_corpus,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
1,NaN,fake,NaN,NaN,NaN,NaN,NaN,NaN,NaN,fake_news_corpus,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
2,NaN,unreliable,NaN,NaN,NaN,NaN,NaN,NaN,NaN,fake_news_corpus,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
3,NaN,unreliable,NaN,NaN,NaN,NaN,NaN,NaN,NaN,fake_news_corpus,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
4,NaN,clickbait,NaN,NaN,NaN,NaN,NaN,NaN,NaN,fake_news_corpus,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3


In [4]:
grouped = df.groupby('dataset')

# Filter out groups where 'data_column' is completely NaN
groups_with_data = grouped.filter(lambda x: x['date'].notna().any())

# Extract distinct categories with data
categories_with_data = groups_with_data['dataset'].unique()

print(categories_with_data)

['fa_kes' 'banfakenews' 'isot_fake_news' 'ti_cnn']


In [5]:
# Print number of entries for each dataset that has date information
for category in categories_with_data:
    count = len(groups_with_data[groups_with_data['dataset'] == category])
    print(f"{category}: {count} entries")


fa_kes: 804 entries
banfakenews: 58478 entries
isot_fake_news: 44898 entries
ti_cnn: 20015 entries


In [6]:
df_with_date = df[df.dataset.isin(['fa_kes', 'banfakenews', 'isot_fake_news', 'ti_cnn'])]
print(len(df_with_date))

124195


In [7]:
for dset in ['fa_kes', 'banfakenews', 'isot_fake_news', 'ti_cnn']:
  print(dset)

  temp_df = df_with_date[df_with_date.dataset == dset]
#   print(len(temp_df))
  #temp_df.veracity = temp_df.label.map({'FALSE':2, 'TRUE':1, 'MOSTLY FALSE':2, 'MOSTLY TRUE':1})
  temp_df = temp_df[temp_df.veracity != 3]
  temp_df = temp_df[temp_df.veracity.notna()]
  temp_df['veracity'] = temp_df['veracity'].astype(int)
#   print(len(temp_df))
  temp_df = temp_df[temp_df.date.notna()]
#   print(len(temp_df))

  temp_df['dates'] = pd.to_datetime(temp_df['date'], format='mixed')

  # Find the earliest (minimum) date in the column
  earliest_date = temp_df['dates'].min()

  # Calculate the time since the earliest date in days
  temp_df['time_since_first'] = ((temp_df['dates'] - earliest_date).dt.total_seconds() / (60 * 60 * 24)).astype(int)

  # Convert the differences to float values
  temp_df['time_since_first'] = temp_df['time_since_first'].astype(float)
#   print(len(temp_df))

  clf = RandomForestClassifier(max_depth=20, random_state=0)#, class_weight='balanced')

  train, test = train_test_split(temp_df, test_size=0.25, random_state=42)

  clf.fit(train.time_since_first.values.reshape(-1,1), train.veracity.values)
  preds = clf.predict(test.time_since_first.values.reshape(-1,1))
  true = test.veracity.values
  print(classification_report(preds,true, digits=3))

fa_kes
              precision    recall  f1-score   support

           1      0.539     0.574     0.556       108
           2      0.465     0.430     0.447        93

    accuracy                          0.507       201
   macro avg      0.502     0.502     0.501       201
weighted avg      0.505     0.507     0.506       201

banfakenews
              precision    recall  f1-score   support

           1      1.000     1.000     1.000     13967
           2      0.936     1.000     0.967        44

    accuracy                          1.000     14011
   macro avg      0.968     1.000     0.983     14011
weighted avg      1.000     1.000     1.000     14011

isot_fake_news
              precision    recall  f1-score   support

           1      0.874     0.736     0.799      1389
           2      0.937     0.974     0.955      5648

    accuracy                          0.927      7037
   macro avg      0.906     0.855     0.877      7037
weighted avg      0.925     0.927     0.